# **Python 網路爬蟲教學 - 使用 requests 庫**
網路爬蟲(Web Scraper)指的是利用程式到網頁、網路上讀取資料  
以爬蟲的角度來看分類，可以將網頁資料分為三類:
1. 別人希望你爬的: 提供API介面與說明文件，使用可能會收費，但是取得的資料很好處理
2. 不希望你爬的: 可以假裝是瀏覽器來取得資料，但後續要再進行資料清理
3. 不希望你爬的，且具有高度防護的: 使用真人認證、帳密登入、行為監測，需要高度技術來突破

**注意事項:**
1. 爬蟲是一個需要深度了解http與html/javascript原理的程式設計，這邊指提供是粗淺的範例
2. 爬蟲取得的資料，可能具有版權或是隱私權爭議，請自行注意
3. 由於是透過網址取得資料，因此本範例隨時間久遠之後，會因網站關閉、網址改變等原因不能使用


In [ ]:
# ===== 1. 基礎設置與安装套件 =====
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import json
import time
from datetime import datetime, timedelta
from IPython.display import clear_output, display, Image
import warnings
warnings.filterwarnings('ignore')

# 設置中文字體（Colab 環境）
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download
import matplotlib.font_manager as fm
fm.fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
plt.rcParams['font.family'] = 'Taipei Sans TC Beta'

print("✅ 所有套件安裝完成！")

In [ ]:

# ===== 2. 基礎概念：什麼是 HTTP 請求？ =====
print("\n=== HTTP 請求基礎概念 ===")

# HTTP 狀態碼說明
def explain_status_codes():
    """解釋常見的 HTTP 狀態碼"""
    status_codes = {
        200: "成功 - 請求已成功處理",
        404: "找不到 - 請求的資源不存在，主機存在但網址錯誤",
        403: "禁止存取 - 沒有權限存取該資源，被檔了",
        500: "伺服器錯誤 - 伺服器內部發生錯誤，可能有真人認證所以被檔了",
        503: "Service Unavailable，可能主機繁忙，可稍後再試",
        429: "請求過於頻繁 - 被限制存取頻率"
    }

    for code, description in status_codes.items():
        print(f"{code}: {description}")

explain_status_codes()







In [ ]:
# ===== 3. 基本的網路請求 =====
print("\n=== 基本網路請求示範 ===")

def basic_request_example():
    """基本的 GET 請求示範"""
    url = "https://httpbin.org/get"

    try:
        response = requests.get(url)
        print(f"狀態碼: {response.status_code}")
        print(f"回應類型: {response.headers.get('content-type')}")
        print(f"回應內容預覽: {response.text[:200]}...")

        # 檢查請求是否成功
        if response.status_code == 200:
            print("✅ 請求成功！")
        else:
            print("❌ 請求失敗！")

    except requests.exceptions.RequestException as e:
        print(f"請求發生錯誤: {e}")

basic_request_example()


In [ ]:
# ===== 4. User-Agent 和請求標頭的重要性 =====
print("\n=== User-Agent 和請求標頭 ===")

def demonstrate_headers():
    """示範使用不同的請求標頭"""
    url = "https://httpbin.org/user-agent"

    # 不使用 User-Agent
    response1 = requests.get(url)
    print("不使用 User-Agent:")
    print(response1.json())

    # 使用自定義 User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response2 = requests.get(url, headers=headers)
    print("\n使用自定義 User-Agent:")
    print(response2.json())

demonstrate_headers()



In [ ]:
# ===== 5. 處理 JSON 資料 =====
print("\n=== JSON 資料處理 ===")

def json_data_example():
    """示範如何處理 JSON 資料"""
    url = 'https://jsonplaceholder.typicode.com/posts/1'

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            print("JSON 資料結構:")
            print(f"標題: {data['title']}")
            print(f"內容: {data['body'][:50]}...")
            print(f"作者 ID: {data['userId']}")
    except Exception as e:
        print(f"處理 JSON 時發生錯誤: {e}")

json_data_example()



In [ ]:
# ===== 6. 錯誤處理和異常管理 =====
print("\n=== 錯誤處理示範 ===")

def error_handling_example():
    """示範完整的錯誤處理"""
    urls = [
        "https://httpbin.org/status/200",  # 正常
        "https://httpbin.org/status/404",  # 找不到
        "https://invalid-url-12345.com"   # 無效網址
    ]

    for url in urls:
        try:
            print(f"\n測試網址: {url}")
            response = requests.get(url, timeout=5)  # 設置超時時間

            if response.status_code == 200:
                print("✅ 請求成功")
            else:
                print(f"⚠️ 狀態碼: {response.status_code}")

        except requests.exceptions.Timeout:
            print("❌ 請求超時")
        except requests.exceptions.ConnectionError:
            print("❌ 連線錯誤")
        except requests.exceptions.RequestException as e:
            print(f"❌ 請求錯誤: {e}")

error_handling_example()


In [ ]:
# ===== 7. 下載和顯示圖片1 - 有沒有headers的差異 =====
print("\n=== 圖片下載與顯示 ===")

def download_and_display_image(image_url,headers=None):
    """下載並顯示圖片"""

    try:
        if headers:
            print("\n有傳入headers")
            response = requests.get(image_url,headers=headers)
        else:
            print("\n沒有傳入headers")
            response = requests.get(image_url)

        if response.status_code == 200:
            # 將圖片資料轉換為 numpy 陣列
            image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
            image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

            # 轉換色彩空間 (BGR to RGB)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # 顯示圖片
            plt.figure(figsize=(6, 4))
            plt.imshow(image_rgb)
            plt.axis('off')
            plt.title('下載的圖片')
            plt.show()

            print("✅ 圖片下載並顯示成功！")
        else:
            print(f"❌ 請求失敗，狀態碼: {response.status_code}")
    except Exception as e:
        print(f"❌ 圖片處理錯誤: {e}")


# 使用一個穩定的圖片網址
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b3/Oreochromis-niloticus-Nairobi.JPG/1920px-Oreochromis-niloticus-Nairobi.JPG"
image_url = "https://i.imgur.com/djsHhnF.png"
print("同一個的圖片url抓取，比較有沒有headers的差異")

# 沒有header
download_and_display_image(image_url)

# 有header，使用自定義 User-Agent
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
download_and_display_image(image_url,headers=headers)



In [ ]:
# ===== 7. 下載和顯示圖片2 =====


headers = {'Upgrade-Insecure-Requests':'1','user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'}
#response = requests.get('http://gis.taiwan.net.tw/XMLReleaseALL_public/activity_C_f.json')

# Adding verify=False to bypass SSL verification
response = requests.get('https://tour.klcg.gov.tw/data/attractions.json', headers=headers, verify=False)

#文字編碼問題，"\u57fa\u9686\u6d77\u6d0b\u5ee3\u5834"這樣的格式就是unicode沒有被正確編碼
print(response.text)
print(response.json())

#指定編碼
#response.encoding = 'big5'
response.encoding = 'utf-8'
print(response.json())


#如何從json中取值
#response.json()回傳dict字典型態
pic_url=response.json()['attractions'][0]['cover_image']
print(pic_url)

# 如何顯示抓到的照片
resp = requests.get(pic_url, stream=True, verify=False).raw
pic = np.asarray(bytearray(resp.read()), dtype="uint8")
pic = cv2.imdecode(pic, cv2.IMREAD_COLOR)

plt.imshow(pic)
plt.axis('off')
plt.show()

# 如何儲存抓到的照片
with open('pic.png', 'wb') as f:
    f.write(requests.get(pic_url, verify=False).content)
    print("儲存成功，請從左側files下載")


In [ ]:
# ===== 8. CSV 資料處理 =====
print("\n=== CSV 資料處理 ===")

def process_csv_data():
    """處理線上 CSV 資料"""
    # 使用一個可靠的 CSV 資料源
    csv_url = "https://raw.githubusercontent.com/datasets/population/master/data/population.csv"

    try:
        # 直接用 pandas 讀取線上 CSV (推薦使用pandas處理所有類似excel格式的資料)
        df = pd.read_csv(csv_url)
        print("CSV 資料讀取成功！")
        print(f"資料形狀: {df.shape}")
        print("\n前 5 筆資料:")
        display(df.head())

        #find Country code = JPN
        print("抓出JPN關鍵字")
        print(df[df.eq("JPN").any(axis=1)])

        # 簡單的資料分析
        if 'Value' in df.columns:
            print(f"\n人口數據統計:")
            print(f"總記錄數: {len(df)}")
            print(f"最大值: {df['Value'].max():,.0f}")
            print(f"最小值: {df['Value'].min():,.0f}")

    except Exception as e:
        print(f"❌ CSV 處理錯誤: {e}")

process_csv_data()


In [ ]:
# ===== 9. API 參數傳遞 =====
print("\n=== API 參數傳遞 ===")

def api_parameters_example():
    """示範 API 參數的使用"""
    base_url = "https://httpbin.org/get"

    # 方法 1: 直接在 URL 中加入參數 (http的GET方法)
    url_with_params = f"{base_url}?name=Python&level=advanced"
    response1 = requests.get(url_with_params)

    # 方法 2: 使用 params 參數
    params = {
        'name': 'Python',
        'level': 'advanced',
        'language': '中文'
    }
    response2 = requests.get(base_url, params=params)

    print("方法 1 - URL 中的參數:")
    print(response1.json()['args'])

    print("\n方法 2 - params 參數:")
    print(response2.json()['args'])

api_parameters_example()



In [ ]:
# ===== 10. 匯率查詢實例 =====
print("\n=== 匯率查詢實例 ===")

def currency_exchange_example():
    """匯率查詢和歷史資料視覺化"""
    try:
        # 取得當前匯率
        url = "https://api.exchangerate-api.com/v4/latest/TWD"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            print(f"基準貨幣: {data['base']}")
            print(f"更新日期: {data['date']}")

            # 顯示主要貨幣匯率
            major_currencies = ['EUR', 'GBP', 'JPY', 'CNY', 'USD', 'KRW', 'THB']
            print("\n主要貨幣匯率 (以台幣為基準):")
            print("-"*20)
            for currency in major_currencies:
                if currency in data['rates']:
                    print(f"1 TWD = {data['rates'][currency]:.4f} {currency}")
                    print(f"1 {currency} = {1/data['rates'][currency]:.4f} TWD")
                    print("-"*20)

    except Exception as e:
        print(f"❌ 匯率查詢錯誤: {e}")

currency_exchange_example()


In [ ]:
# ===== 11. 網站狀態監控 =====
print("\n=== 網站狀態監控 ===")

def website_monitoring():
    """監控多個網站的狀態"""
    websites = [
        "https://www.google.com",
        "https://www.github.com",
        "https://www.stackoverflow.com"
    ]

    results = []

    for site in websites:
        try:
            start_time = time.time()
            response = requests.get(site, timeout=10)
            response_time = time.time() - start_time

            results.append({
                'Website': site.replace('https://www.', ''),
                'Status': response.status_code,
                'Response Time': f"{response_time:.2f}s",
                'Available': '✅' if response.status_code == 200 else '❌'
            })

        except Exception as e:
            results.append({
                'Website': site.replace('https://www.', ''),
                'Status': 'Error',
                'Response Time': 'N/A',
                'Available': '❌'
            })

    # 建立結果 DataFrame
    df = pd.DataFrame(results)
    print("網站狀態監控結果:")
    display(df)

website_monitoring()


In [ ]:
# ===== 12. 應用: 即時路況影像 - 照片模式 =====
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import randn
from time import sleep

#高速公路監視攝影機
cam_url='https://cctvn.freeway.gov.tw/abs2mjpg/jpg?camera=10000'

for i in range(5):
  resp = requests.get(cam_url, stream=True).raw
  pic = np.asarray(bytearray(resp.read()), dtype="uint8")
  pic = cv2.imdecode(pic, cv2.IMREAD_COLOR)

  clear_output()
  plt.imshow(pic)
  plt.show()
  sleep(2)



In [ ]:
# ===== 12. 應用2: 即時路況影像 - 影片模式 =====

#基隆路口監視器
cam_url = "https://cctv.klcg.gov.tw/47668ccf"

cap = cv2.VideoCapture(cam_url)

if not cap.isOpened():
    print("無法打開影片串流")
else:
  for i in range(20):
    ret, frame = cap.read()
    if ret:
        # OpenCV 讀取的影像是 BGR 格式，需要轉換為 RGB 才能用 matplotlib 顯示
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.imshow(frame_rgb)
        plt.show()
    else:
        print("無法讀取影像幀")

  cap.release()

**=== 爬蟲禮儀和最佳實踐 ===**
1. 尊重 robots.txt: 檢查網站的 robots.txt 檔案，遵守爬蟲規則
2. 適當的延遲: 在請求之間加入延遲，避免對伺服器造成過大負擔
3. 使用適當的 User-Agent: 不要偽裝成其他爬蟲或瀏覽器
4. 處理錯誤: 妥善處理網路錯誤和異常狀況
5. 遵守法律: 確保爬蟲行為符合法律規範和網站條款
6. 快取機制: 避免重複請求相同的資料

In [ ]:
# ===== 14. 簡單的延遲和頻率控制 =====
def polite_crawler_example():
    """有禮貌的爬蟲示範"""
    urls = [
        "https://httpbin.org/delay/1",
        "https://httpbin.org/delay/1",
        "https://pytorch.org/"
    ]

    print("\n=== 有禮貌的爬蟲示範 ===")

    for i, url in enumerate(urls, 1):
        print(f"正在請求第 {i} 個網址...")

        try:
            start_time = time.time()
            response = requests.get(url, timeout=10)
            end_time = time.time()

            print(f"✅ 完成，耗時: {end_time - start_time:.2f} 秒")

            # 在請求之間加入延遲（好習慣）
            if i < len(urls):
                print("⏳ 等待 2 秒...")
                time.sleep(2)

        except Exception as e:
            print(f"❌ 錯誤: {e}")

polite_crawler_example()


In [ ]:

# ===== 13. Session 的使用 =====
print("\n=== Session 使用示範 ===")

def session_example():
    """示範 Session 的使用（保持連線和 cookies）"""
    # 建立 session
    session = requests.Session()

    # 設置 session 的預設標頭
    session.headers.update({
        'User-Agent': 'Python Learning Bot 1.0'
    })

    # 使用 session 進行多次請求
    urls = [
        "https://httpbin.org/headers",
        "https://httpbin.org/user-agent"
    ]

    for url in urls:
        try:
            response = session.get(url)
            if response.status_code == 200:
                data = response.json()
                print(f"URL: {url}")
                if 'headers' in data:
                    print(f"User-Agent: {data['headers'].get('User-Agent', 'Not found')}")
                elif 'user-agent' in data:
                    print(f"User-Agent: {data['user-agent']}")
                print("-" * 50)
        except Exception as e:
            print(f"錯誤: {e}")

    # 關閉 session
    session.close()

session_example()


**=== 學習總結 ===**

📚 核心概念:
  • HTTP 請求和回應的基本概念  
  • 狀態碼的意義和處理方式  
  • JSON 資料格式的解析和使用  
  • 錯誤處理和異常管理  
  
📚 實用技巧:  
  • 使用適當的 User-Agent 和請求標頭  
  • 參數傳遞的不同方法  
  • 圖片和檔案的下載處理  
  • Session 的使用時機  
  
📚 最佳實踐:  
  • 遵守爬蟲禮儀    
  • 實施適當的延遲控制  
  • 完善的錯誤處理機制  
  • 資料的有效儲存和處理  
  
🎉 恭喜！您已經完成了 Python 網路爬蟲的基礎學習！  
💡 建議下一步學習: BeautifulSoup (HTML 解析) 和 Selenium (動態網頁爬取)  